# Genesis4 Particles

This shows examples of the various ways to input particle data into Genesis4.

In [ ]:
import logging
import os
from math import pi, sqrt

import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import c

import genesis.version4 as g4

logging.basicConfig()
# logging.getLogger("genesis").setLevel("DEBUG")

%config InlineBackend.figure_format = 'retina'

# Lattice

Create a simple drift lattice

In [ ]:
D1 = g4.Drift(L=1)
lattice = g4.Lattice(elements={"D1": D1, "LAT": g4.Line(elements=[D1])})

# `profile_gauss`

This profile will make a Gaussian distribition. Here we do some calculations to make the correct bunch length for a given bunch charge to provide a peak current.

In [ ]:
PEAK_CURRENT = 1000
BUNCH_CHARGE = 100e-12
SIGMA_T = BUNCH_CHARGE / (sqrt(2 * pi) * PEAK_CURRENT)
SIGMA_Z = SIGMA_T * c
SLEN = 6 * SIGMA_Z
S0 = 3 * SIGMA_Z
SIGMA_T, SIGMA_Z, SLEN

In [ ]:
main = g4.MainInput(
    namelists=[
        g4.Setup(
            rootname="drift_test",
            # lattice="LATFILE",
            beamline="LAT",
            gamma0=1000,
            lambda0=1e-07,
            delz=0.026,
            seed=123456,
            npart=128,
        ),
        g4.Time(slen=SLEN),
        g4.ProfileGauss(
            label="beamcurrent",
            c0=PEAK_CURRENT,
            s0=S0,
            sig=SIGMA_Z,
        ),
        g4.Beam(
            gamma=1000,
            delgam=1,
            current="beamcurrent",
        ),
        g4.Track(zstop=1),
        g4.Write(beam="end"),
    ],
)

G = g4.Genesis4(main, lattice, verbose=True)
output = G.run()

In [ ]:
G.input.main.setup.delz

In [ ]:
print(G.output.run.output_log)

In [ ]:
output.load_particles()
P1 = output.particles["end"]
P1.drift_to_z()
P1.plot("t", "energy")
P1

In [ ]:
output.particles["end"]

Check the charge

In [ ]:
P1.charge

# `profile_file`

LUME-Genesis automatically makes an HDF5 file with `ProfileArray`.

In [ ]:
NPTS = 100
SLEN = 100e-6
S = np.linspace(0, SLEN, NPTS)
CURRENT = np.linspace(1, 1000.0, NPTS)
plt.plot(S, CURRENT)

In [ ]:
main = g4.MainInput(
    namelists=[
        g4.Setup(
            rootname="drift_test",
            # lattice=lattice,
            beamline="LAT",
            gamma0=1000,
            lambda0=1e-07,
            delz=0.026,
            seed=123456,
            npart=128,
        ),
        g4.Time(slen=SLEN),
        g4.ProfileArray(label="beamcurrent", xdata=S, ydata=CURRENT),
        g4.Beam(
            gamma=1000,
            delgam=1,
            current="beamcurrent",
            ex=1e-06,
            ey=1e-06,
            betax=7.910909406464387,
            betay=16.881178621346898,
            alphax=-0.7393217413918415,
            alphay=1.3870723536888105,
        ),
        g4.Track(zstop=1),
        g4.Write(beam="end"),
    ]
)

G = g4.Genesis4(main, lattice, verbose=True)
output = G.run()

### Inspect the input and output

In [ ]:
print(main.to_genesis())

In [ ]:
print(lattice.to_genesis())

In [ ]:
print(output.run.output_log)

In [ ]:
output.meta

In [ ]:
output.load_particles()
P1 = output.particles["end"]
P1.drift_to_z()
P1.plot("t", "energy")
P1

Resample particles for equal weights. This is neccessary when reading from a distribution file.

In [ ]:
NSAMPLE = len(P1)
P1r = P1.resample(NSAMPLE)
P1r.plot("t", "energy")
P1r

Make a more interesting distribution from this:

In [ ]:
P1r.pz[0 : len(P1) // 2] *= 1.1
P1r.plot("t", "energy")

ParticleGroup can write to a file for Genesis4.

**Please note** that LUME-Genesis will write the distribution for you prior to running Genesis4, so this step is not necessary.

In [ ]:
DIST_FILE = "genesis4_distribution.h5"
P1r.write_genesis4_distribution(DIST_FILE, verbose=True)

# ParticleGroup

Genesis4Input directly supports OpenPMD-beamphysics `ParticleGroup` instances.
When using the `MainInput.initial_particles` property setter, LUME-Genesis will ensure the namelist is added before the first "Track" or "Write" namelist in the main input.

It will implicitly set the `import_distribution` charge and the `time.slen` to the calculated time window from the particles, equivalent to the following:
```python
    import_distribution.charge = particles.charge
    main.time.slen = max(
        c_light * np.ptp(particles.t),
        np.ptp(particles.z),
    )
```

Additionally, the appropriate input file for Genesis4 will be written automatically when Genesis4 is executed.

In [ ]:
main = g4.MainInput(
    namelists=[
        g4.Setup(
            rootname="drift_test",
            # lattice=full_path(LATFILE),
            beamline="LAT",
            gamma0=1000,
            lambda0=1e-07,
            delz=0.026,
            seed=123456,
            npart=512,
        ),
        g4.Time(),
        g4.Track(zstop=1),
        g4.Write(beam="end"),
    ],
)

G1 = g4.Genesis4(main, lattice, verbose=True, initial_particles=P1r)
output = G1.run()

In [ ]:
output.run

In [ ]:
output.load_particles()
P2 = output.particles["end"]
P2.z

In [ ]:
P2.drift_to_z()
P2.plot("t", "energy")
P2

In [ ]:
P2.plot("weight", bins=100)

Notice that `importdistribution` is filled in:

In [ ]:
print(G1.input.to_genesis())

# Cleanup

In [ ]:
G1.input.initial_particles

In [ ]:
os.remove(DIST_FILE)